# Spark机器学习基础
by [寒小阳](http://blog.csdn.net/han_xiaoyang) (hanxiaoyang.ml@gmail.com)

## 无监督学习

### 0.K-means

In [1]:
from __future__ import print_function
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql import SparkSession

In [2]:
! head -5 data/mllib/sample_kmeans_data.txt

0 1:0.0 2:0.0 3:0.0
1 1:0.1 2:0.1 3:0.1
2 1:0.2 2:0.2 3:0.2
3 1:9.0 2:9.0 3:9.0
4 1:9.1 2:9.1 3:9.1


In [3]:
spark = SparkSession\
        .builder\
        .appName("KMeansExample")\
        .getOrCreate()

dataset = spark.read.format("libsvm").load("data/mllib/sample_kmeans_data.txt")

# 训练K-means聚类模型
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)

# 预测(即分配聚类中心)
predictions = model.transform(dataset)

# 根据Silhouette得分评估(pyspark2.2里新加)
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# 输出预测结果
print("predicted Center: ")
for center in predictions[['prediction']].collect():
    print(center.asDict())

# 聚类中心
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

spark.stop()

Silhouette with squared euclidean distance = 0.9997530305375207
predicted Center: 
{'prediction': 0}
{'prediction': 0}
{'prediction': 0}
{'prediction': 1}
{'prediction': 1}
{'prediction': 1}
Cluster Centers: 
[ 0.1  0.1  0.1]
[ 9.1  9.1  9.1]


### 2.GMM模型

In [4]:
from __future__ import print_function
from pyspark.ml.clustering import GaussianMixture
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession\
        .builder\
        .appName("GaussianMixtureExample")\
        .getOrCreate()

dataset = spark.read.format("libsvm").load("data/mllib/sample_kmeans_data.txt")

gmm = GaussianMixture().setK(2).setSeed(0)
model = gmm.fit(dataset)

print("Gaussians shown as a DataFrame: ")
model.gaussiansDF.show(truncate=False)

spark.stop()

Gaussians shown as a DataFrame: 
+-------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|mean                                                         |cov                                                                                                                                                                                                     |
+-------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[9.099999999999984,9.099999999999984,9.099999999999984]      |0.006666666666812185  0.006666666666812185  0.006666666666812185  
0.006666666666812185  0.006666666666812185

### 3.关联规则
我这里是pyspark 2.2以下的版本的写法，新版可以参考此程序之下的程序

In [6]:
!head -5 data/mllib/sample_fpgrowth.txt

r z h k p
z y x w v u t s
s x o n r
x z y m t s q e
z


In [7]:
from pyspark.mllib.fpm import FPGrowth
from pyspark.sql import SparkSession
spark = SparkSession\
        .builder\
        .appName("FPGrowthExample")\
        .getOrCreate()

data = spark.sparkContext.textFile("data/mllib/sample_fpgrowth.txt")
transactions = data.map(lambda line: line.strip().split(' '))
model = FPGrowth.train(transactions, minSupport=0.2, numPartitions=10)
result = model.freqItemsets().collect()
for fi in result:
    print(fi)

spark.stop()

FreqItemset(items=['z'], freq=5)
FreqItemset(items=['x'], freq=4)
FreqItemset(items=['x', 'z'], freq=3)
FreqItemset(items=['y'], freq=3)
FreqItemset(items=['y', 'x'], freq=3)
FreqItemset(items=['y', 'x', 'z'], freq=3)
FreqItemset(items=['y', 'z'], freq=3)
FreqItemset(items=['r'], freq=3)
FreqItemset(items=['r', 'x'], freq=2)
FreqItemset(items=['r', 'z'], freq=2)
FreqItemset(items=['s'], freq=3)
FreqItemset(items=['s', 'y'], freq=2)
FreqItemset(items=['s', 'y', 'x'], freq=2)
FreqItemset(items=['s', 'y', 'x', 'z'], freq=2)
FreqItemset(items=['s', 'y', 'z'], freq=2)
FreqItemset(items=['s', 'x'], freq=3)
FreqItemset(items=['s', 'x', 'z'], freq=2)
FreqItemset(items=['s', 'z'], freq=2)
FreqItemset(items=['t'], freq=3)
FreqItemset(items=['t', 'y'], freq=3)
FreqItemset(items=['t', 'y', 'x'], freq=3)
FreqItemset(items=['t', 'y', 'x', 'z'], freq=3)
FreqItemset(items=['t', 'y', 'z'], freq=3)
FreqItemset(items=['t', 's'], freq=2)
FreqItemset(items=['t', 's', 'y'], freq=2)
FreqItemset(items=['t', '

In [8]:
from pyspark.ml.fpm import FPGrowth
spark = SparkSession\
        .builder\
        .appName("FPGrowthExample")\
        .getOrCreate()

df = spark.createDataFrame([
    (0, [1, 2, 5]),
    (1, [1, 2, 3, 5]),
    (2, [1, 2])
], ["id", "items"])

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.5, minConfidence=0.6)
model = fpGrowth.fit(df)

# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(df).show()

spark.stop()

+---------+----+
|    items|freq|
+---------+----+
|      [1]|   3|
|      [2]|   3|
|   [2, 1]|   3|
|      [5]|   2|
|   [5, 2]|   2|
|[5, 2, 1]|   2|
|   [5, 1]|   2|
+---------+----+

+----------+----------+------------------+
|antecedent|consequent|        confidence|
+----------+----------+------------------+
|    [5, 2]|       [1]|               1.0|
|       [2]|       [1]|               1.0|
|       [2]|       [5]|0.6666666666666666|
|    [2, 1]|       [5]|0.6666666666666666|
|       [5]|       [2]|               1.0|
|       [5]|       [1]|               1.0|
|    [5, 1]|       [2]|               1.0|
|       [1]|       [2]|               1.0|
|       [1]|       [5]|0.6666666666666666|
+----------+----------+------------------+

+---+------------+----------+
| id|       items|prediction|
+---+------------+----------+
|  0|   [1, 2, 5]|        []|
|  1|[1, 2, 3, 5]|        []|
|  2|      [1, 2]|       [5]|
+---+------------+----------+



### 4.LDA主题模型

In [9]:
from __future__ import print_function
from pyspark.ml.clustering import LDA
from pyspark.sql import SparkSession

In [10]:
! head -5 data/mllib/sample_lda_libsvm_data.txt

0 1:1 2:2 3:6 4:0 5:2 6:3 7:1 8:1 9:0 10:0 11:3
1 1:1 2:3 3:0 4:1 5:3 6:0 7:0 8:2 9:0 10:0 11:1
2 1:1 2:4 3:1 4:0 5:0 6:4 7:9 8:0 9:1 10:2 11:0
3 1:2 2:1 3:0 4:3 5:0 6:0 7:5 8:0 9:2 10:3 11:9
4 1:3 2:1 3:1 4:9 5:3 6:0 7:2 8:0 9:0 10:1 11:3


In [11]:
spark = SparkSession \
        .builder \
        .appName("LDAExample") \
        .getOrCreate()

# 加载数据
dataset = spark.read.format("libsvm").load("data/mllib/sample_lda_libsvm_data.txt")

# 训练LDA模型
lda = LDA(k=10, maxIter=10)
model = lda.fit(dataset)

ll = model.logLikelihood(dataset)
lp = model.logPerplexity(dataset)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp)+"\n")

# 输出主题
topics = model.describeTopics(3)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)

# 数据集解析
print("transform dataset:\n")
transformed = model.transform(dataset)
transformed.show(truncate=False)

spark.stop()

The lower bound on the log likelihood of the entire corpus: -784.563312043982
The upper bound on perplexity: 3.0175511876689556

The topics described by their top-weighted terms:
+-----+-----------+---------------------------------------------------------------+
|topic|termIndices|termWeights                                                    |
+-----+-----------+---------------------------------------------------------------+
|0    |[3, 4, 10] |[0.30479149989901694, 0.12278796514250867, 0.12161706973908994]|
|1    |[10, 5, 1] |[0.10472118214638892, 0.10407538205701086, 0.09443251272060076]|
|2    |[0, 9, 8]  |[0.10725946461445685, 0.10401349116870545, 0.09445471149622425]|
|3    |[5, 0, 4]  |[0.1861359631937542, 0.15982888484836216, 0.15787541409980693] |
|4    |[6, 1, 9]  |[0.22100346160439127, 0.1886216316667507, 0.13487077710226394] |
|5    |[0, 4, 8]  |[0.11475105421219843, 0.0938293926323442, 0.09373655278318598] |
|6    |[10, 0, 1] |[0.09936287293318637, 0.09919070503430866, 0.0

### PCA降维

In [12]:
from __future__ import print_function
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession

In [13]:
spark = SparkSession\
        .builder\
        .appName("PCAExample")\
        .getOrCreate()

# 构建一份fake data
data = [(Vectors.sparse(5, [(1, 1.0), (3, 7.0)]),),
        (Vectors.dense([2.0, 0.0, 3.0, 4.0, 5.0]),),
        (Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),)]
df = spark.createDataFrame(data, ["features"])

# PCA降维
pca = PCA(k=3, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(df)

result = model.transform(df).select("pcaFeatures")
result.show(truncate=False)

spark.stop()

+-----------------------------------------------------------+
|pcaFeatures                                                |
+-----------------------------------------------------------+
|[1.6485728230883807,-4.013282700516296,-5.524543751369388] |
|[-4.645104331781534,-1.1167972663619026,-5.524543751369387]|
|[-6.428880535676489,-5.337951427775355,-5.524543751369389] |
+-----------------------------------------------------------+



### word2vec词嵌入

In [14]:
from __future__ import print_function
from pyspark.ml.feature import Word2Vec
from pyspark.sql import SparkSession

In [15]:
spark = SparkSession\
        .builder\
        .appName("Word2VecExample")\
        .getOrCreate()

# 输入是bag of words形式
documentDF = spark.createDataFrame([
    ("Hi I heard about Spark".split(" "), ),
    ("I wish Java could use case classes".split(" "), ),
    ("Logistic regression models are neat".split(" "), )
], ["text"])

# 设置窗口长度等参数，词嵌入学习
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result")
model = word2Vec.fit(documentDF)

# 输出词和词向量
model.getVectors().show()

result = model.transform(documentDF)
for row in result.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

spark.stop()

+----------+--------------------+
|      word|              vector|
+----------+--------------------+
|     heard|[-0.1366093307733...|
|       are|[-0.1653077900409...|
|      neat|[0.09378280490636...|
|   classes|[-0.1547942310571...|
|         I|[-0.0424778945744...|
|regression|[-0.0461251139640...|
|  Logistic|[0.02324013225734...|
|     Spark|[-0.0981360152363...|
|     could|[-0.0302416980266...|
|       use|[0.02537945471704...|
|        Hi|[-0.0277608968317...|
|    models|[-0.1365544795989...|
|      case|[-0.1623256206512...|
|     about|[0.03644379600882...|
|      Java|[-0.1164701506495...|
|      wish|[-0.0754781961441...|
+----------+--------------------+

Text: [Hi, I, heard, about, Spark] => 
Vector: [-0.0537080682814,0.0383701570332,0.0819620683789]

Text: [I, wish, Java, could, use, case, classes] => 
Vector: [-0.079486905198,0.0158437477159,0.0428948812187]

Text: [Logistic, regression, models, are, neat] => 
Vector: [-0.0461928892881,0.0157909940928,-0.05168700143